<a href="https://colab.research.google.com/github/EkeminiUmanah/g05-used-cars/blob/EkeminiUmanah-patch-2/UsedVehiclesPrediction1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#google.colab import drive
#drive.mount('/content/drive')

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Hamoye Stage G Used_Vehicles/vehiclesdf_NaNDropped.csv')
#In this dataset, The remaining NaN values were dropped after filling missing values in categorical features with mode

In [5]:
df.isnull().sum()

price           0
manufacturer    0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
title_status    0
transmission    0
drive           0
type            0
paint_color     0
car_age         0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96886 entries, 0 to 96885
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         96886 non-null  int64  
 1   manufacturer  96886 non-null  object 
 2   model         96886 non-null  object 
 3   condition     96886 non-null  object 
 4   cylinders     96886 non-null  object 
 5   fuel          96886 non-null  object 
 6   odometer      96886 non-null  float64
 7   title_status  96886 non-null  object 
 8   transmission  96886 non-null  object 
 9   drive         96886 non-null  object 
 10  type          96886 non-null  object 
 11  paint_color   96886 non-null  object 
 12  car_age       96886 non-null  float64
dtypes: float64(2), int64(1), object(10)
memory usage: 9.6+ MB


In [7]:
df.corr()

,price,odometer,car_age
price,1.000000,-0.56548,-0.603276
odometer,-0.565480,1.00000,0.572180
car_age,-0.603276,0.57218,1.000000


After data preprocessing, we can now see a negative 0.56548 correlation of odometer with price, and a positive 0.572180correlation of car_age with price

##### Converting Categorical Variables to numerical:

In [8]:
df.columns

Index(['price', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'title_status', 'transmission', 'drive', 'type',
       'paint_color', 'car_age'],
      dtype='object')

In [9]:
encoded_data = pd.get_dummies(df[['fuel', 'transmission', 'drive', 'type', 'paint_color']], drop_first=True)
encoded_data.head()

,fuel_electric,fuel_gas,fuel_hybrid,fuel_other,transmission_manual,transmission_other,drive_fwd,drive_rwd,type_bus,type_convertible,type_coupe,type_hatchback,type_mini-van,type_offroad,type_other,type_pickup,type_sedan,type_truck,type_van,type_wagon,paint_color_blue,paint_color_brown,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_white,paint_color_yellow
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


encoded_data = pd.concat([df[['price', 'odometer', 'car_age', 'manufacturer', 'model', 'cylinders', 'condition', 'title_status']].apply(le.fit_transform), encoded_data], axis = 1)
encoded_data.head()

,price,odometer,car_age,manufacturer,model,cylinders,condition,title_status,fuel_electric,fuel_gas,fuel_hybrid,fuel_other,transmission_manual,transmission_other,drive_fwd,drive_rwd,type_bus,type_convertible,type_coupe,type_hatchback,type_mini-van,type_offroad,type_other,type_pickup,type_sedan,type_truck,type_van,type_wagon,paint_color_blue,paint_color_brown,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_white,paint_color_yellow
0,1001,39509,20,12,6939,6,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,3883,45524,13,12,8606,6,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,2927,42175,8,11,4180,5,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
3,1248,26023,10,7,3753,3,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,1529,40993,9,7,9883,5,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


### Normalizing our Data

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X = encoded_data.drop(['price'], axis = 1)
X = scaler.fit_transform(X)

y = encoded_data['price']

#### Splitting our Data:

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

### Creating Our Model:

In [15]:
!pip install catboost

In [16]:
# importing the required algorithms
from sklearn.linear_model import Ridge, Lasso, LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor

# import evaluation metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# create a dictionary of the algorithms to be used:
models = {'KNeighborsRegressor':KNeighborsRegressor(),
        'RandomForestRegressor':RandomForestRegressor(),
        'ExtraTreesRegressor':ExtraTreesRegressor(),
        'GradientBoostingRegressor':GradientBoostingRegressor(),
        'DecisionTreeRegressor':DecisionTreeRegressor(),
        'CatBoostRegressor': CatBoostRegressor()}


In [17]:
#Funtion to compute RMSE, MAE, MSE, R2 scores

def pipeline(models, X_train, X_test, y_train, y_test):
    
    # store properties of each model
    prop = []
    
    # loop through the dictionary of models
    for reg_name, regressor in models.items():
        # empty dict for storing properties of each models
        reg_dict = {}
        # store the name of each model
        reg_dict['Name'] = reg_name
        # fit the regressor model
        regressor.fit(X_train, y_train)
        # compute r squared score
        reg_dict['r2'] = round(r2_score(y_test, regressor.predict(X_test)), 3)
        # compute the mean_absolute_error
        reg_dict['MAE'] = round(mean_absolute_error(y_test, regressor.predict(X_test)), 3)
        #compute the mean_squared_error
        reg_dict['MSE'] = round(mean_squared_error(y_test, regressor.predict(X_test)), 3)
        # compute the RMSE
        reg_dict['RMSE'] = round(np.sqrt(mean_squared_error(y_test, regressor.predict(X_test))), 3)
        # append the properties of a single regressor to the prop list
        prop.append(reg_dict)
     
    # create a dataframe with a list of all the model properties
    summary_df = pd.DataFrame(prop)
   
        
        
    return summary_df
        

In [18]:
# execute the function
pipeline(models, X_train, X_test, y_train, y_test)

Learning rate set to 0.083698
0:	learn: 1906.6243434	total: 60.2ms	remaining: 1m
1:	learn: 1804.3461424	total: 71.3ms	remaining: 35.6s
2:	learn: 1712.6477950	total: 82.2ms	remaining: 27.3s
3:	learn: 1633.2324360	total: 92.8ms	remaining: 23.1s
4:	learn: 1558.4646984	total: 104ms	remaining: 20.6s
5:	learn: 1493.2112591	total: 115ms	remaining: 19s
6:	learn: 1434.4459779	total: 125ms	remaining: 17.8s
7:	learn: 1382.9320936	total: 136ms	remaining: 16.9s
8:	learn: 1336.3373371	total: 147ms	remaining: 16.2s
9:	learn: 1293.9051118	total: 158ms	remaining: 15.6s
10:	learn: 1255.6741746	total: 168ms	remaining: 15.1s
11:	learn: 1222.9995648	total: 179ms	remaining: 14.7s
12:	learn: 1194.1903209	total: 195ms	remaining: 14.8s
13:	learn: 1166.8324826	total: 206ms	remaining: 14.5s
14:	learn: 1142.8585377	total: 216ms	remaining: 14.2s
15:	learn: 1121.4906270	total: 227ms	remaining: 14s
16:	learn: 1102.5070574	total: 237ms	remaining: 13.7s
17:	learn: 1085.7982380	total: 248ms	remaining: 13.5s
18:	learn: 

,Name,r2,MAE,MSE,RMSE
0,KNeighborsRegressor,0.763,665.532,964112.669,981.892
1,RandomForestRegressor,0.875,453.670,507671.082,712.510
2,ExtraTreesRegressor,0.863,469.781,557153.802,746.427
3,GradientBoostingRegressor,0.813,612.101,760989.128,872.347
4,DecisionTreeRegressor,0.774,588.734,918691.278,958.484
5,CatBoostRegressor,0.882,463.797,481758.709,694.088


In [19]:
import gc

In [20]:
gc.collect()

78

##### Hyperparameter tuning for the best performing model(Catboost Regressor):

In [23]:
from sklearn.model_selection import GridSearchCV

parameters = {'depth'         : [6,8,10,12,14,16,18,20],
              'learning_rate' : [0.01, 0.05, 0.1, 0.5, 1],
              'iterations'    : [30, 50, 100]
              }
cb_model = CatBoostRegressor()
grid_search = GridSearchCV(cb_model, parameters, cv=2, n_jobs=-1)

In [24]:
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0:	learn: 1344.5043910	total: 32.5ms	remaining: 3.22s
1:	learn: 1076.1783507	total: 58.3ms	remaining: 2.85s
2:	learn: 964.5497215	total: 83.1ms	remaining: 2.69s
3:	learn: 906.2068041	total: 107ms	remaining: 2.58s
4:	learn: 875.1315132	total: 132ms	remaining: 2.51s
5:	learn: 852.5762455	total: 156ms	remaining: 2.45s
6:	learn: 840.6992591	total: 181ms	remaining: 2.41s
7:	learn: 832.3933650	total: 206ms	remaining: 2.37s
8:	learn: 819.6337252	total: 230ms	remaining: 2.33s
9:	learn: 806.7330288	total: 266ms	remaining: 2.4s
10:	learn: 796.0480727	total: 297ms	remaining: 2.4s
11:	learn: 791.0885223	total: 321ms	remaining: 2.35s
12:	learn: 787.0015487	total: 345ms	remaining: 2.31s
13:	learn: 781.0693871	total: 370ms	remaining: 2.27s
14:	learn: 773.5102547	total: 395ms	remaining: 2.24s
15:	learn: 769.3908497	total: 420ms	remaining: 2.2s
16:	learn: 762.6079581	total: 445ms	remaining: 2.17s
17:	learn: 757.8509231	total: 474ms	remaining: 2.16s
18:	learn: 751.9079355	total: 499ms	remaining: 2.13s
1

GridSearchCV(cv=2, error_score=nan,
             estimator=<catboost.core.CatBoostRegressor object at 0x7faff5f728d0>,
             iid='deprecated', n_jobs=-1,
             param_grid={'depth': [6, 8, 10, 12, 14, 16, 18, 20],
                         'iterations': [30, 50, 100],
                         'learning_rate': [0.01, 0.05, 0.1, 0.5, 1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [25]:
gc.collect()

1281

In [26]:
# get best params
grid_search.best_params_

{'depth': 10, 'iterations': 100, 'learning_rate': 0.5}

In [27]:
# get best model
model = grid_search.best_estimator_

In [28]:
model

In [31]:
print(f'R2_score on training_set with tuned parameters: {model.score(X_train, y_train)}')
print(f'R2_score on testing_set with tuned parameters: {round(model.score(X_test, y_test), 3)}')
print(f'MSE_score on testing_set with tuned parameters: {round(mean_squared_error(y_test, model.predict(X_test)), 3)}')
print(f'RMSE_score on testing_set with tuned parameters: {round(np.sqrt(mean_squared_error(y_test, model.predict(X_test))), 3)}')
print(f'MAE_score with tuned parameters: {round(mean_absolute_error(y_test, model.predict(X_test)), 3)}')

R2_score on training_set with tuned parameters: 0.9150190506431892
R2_score on testing_set with tuned parameters: 0.881
MSE_score on testing_set with tuned parameters: 483583.114
RMSE_score on testing_set with tuned parameters: 695.401
MAE_score with tuned parameters: 463.236
